In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

## Titanic
data_titanic = pd.read_csv("data/titanic/train.csv")

column_titanic_dict = {"PassengerId": "PassengerId", "Survived": "Survived", "Pclass": "TicketClass", "Name": "Name", "Sex": "Sex", "Age": "Age", "SibSp": "NumberSiblingsSpouses", "Parch": "NumberParentsChildren", "Ticket": "TicketNumber", "Fare": "Fare", "Cabin": "CabinNumber", "Embarked": "Port"}
data_titanic = data_titanic.rename(columns=column_titanic_dict)

train_X_titanic_columns = ["TicketClass", "Sex", "Age", "NumberSiblingsSpouses", "NumberParentsChildren", "Fare", "Port"]
train_y_titanic_columns = ["Survived"]
train_X_titanic_ordinal_columns = ["Sex", "Port"]

X_titanic = data_titanic[train_X_titanic_columns]
y_titanic = data_titanic[train_y_titanic_columns]
 
X_titanic.loc[:, train_X_titanic_ordinal_columns] = OrdinalEncoder().fit_transform(X_titanic[train_X_titanic_ordinal_columns])

X_titanic_train, X_titanic_test, y_titanic_train, y_titanic_test = train_test_split(X_titanic, y_titanic, test_size=0.4)
y_titanic_train = np.squeeze(y_titanic_train)
y_titanic_test = np.squeeze(y_titanic_test)

## Machine Failure
data_machine_failure = pd.read_csv("data/machine_failure/train.csv")

column_machine_failure_dict = {"PassengerId": "PassengerId", "Survived": "Survived", "Pclass": "TicketClass", "Name": "Name", "Sex": "Sex", "Age": "Age", "SibSp": "NumberSiblingsSpouses", "Parch": "NumberParentsChildren", "Ticket": "TicketNumber", "Fare": "Fare", "Cabin": "CabinNumber", "Embarked": "Port"}
data_machine_failure = data_machine_failure.rename(columns=column_machine_failure_dict)

train_X_machine_failure_columns = ["TicketClass", "Sex", "Age", "NumberSiblingsSpouses", "NumberParentsChildren", "Fare", "Port"]
train_y_machine_failure_columns = ["Survived"]
train_X_machine_failure_ordinal_columns = ["Sex", "Port"]

X_machine_failure = data_machine_failure[train_X_machine_failure_columns]
y_machine_failure = data_machine_failure[train_y_machine_failure_columns]
 
X_machine_failure.loc[:, train_X_machine_failure_ordinal_columns] = OrdinalEncoder().fit_transform(X_machine_failure[train_X_machine_failure_ordinal_columns])

X_machine_failure_train, X_machine_failure_test, y_machine_failure_train, y_machine_failure_test = train_test_split(X_machine_failure, y_machine_failure, test_size=0.4)
y_machine_failure_train = np.squeeze(y_machine_failure_train)
y_machine_failure_test = np.squeeze(y_machine_failure_test)

## Synthetic
X_synthetic = np.random.randn(int(3e3)).reshape([int(1e3), 3])
y_synthetic = X_synthetic[:, 0] + X_synthetic[:, 1] + X_synthetic[:, 2] > 0
yn_synthetic = np.logical_not(y_synthetic)

X_synthetic[y_synthetic, :] += np.array([1, 1, 1]) * 0.1
X_synthetic[yn_synthetic, :] -= np.array([1, 1, 1]) * 0.1

X_synthetic_train, X_synthetic_test, y_synthetic_train, y_synthetic_test = train_test_split(X_synthetic, y_synthetic, test_size=0.4)
y_synthetic_train = np.squeeze(y_synthetic_train)
y_synthetic_test = np.squeeze(y_synthetic_test)

/var/folders/3f/d8gl3k3j0bn3wk_cqjvfm31w0000gn/T/ipykernel_17443/907899923.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_titanic.loc[:, train_X_titanic_ordinal_columns] = OrdinalEncoder().fit_transform(X_titanic[train_X_titanic_ordinal_columns])
/var/folders/3f/d8gl3k3j0bn3wk_cqjvfm31w0000gn/T/ipykernel_17443/907899923.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_machine_failure.loc[:, train_X_machine_failure_ordinal_columns] = OrdinalEncoder().fit_transform(X_machine_failure[train_X_mach

In [9]:
## Standard Scaler

import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score, recall_score

## Titanic
print("Titanic")
# K Nearest Neighbors Classifier
print("Optimize K Nearest Neighbors Classifier")
knn_clf_param_grid = [
  {"knn_clf__n_neighbors": [5, 10, 15, 20], "knn_clf__weights": ["uniform", "distance"]},
]
knn_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ("knn_clf", KNeighborsClassifier()),]),
    knn_clf_param_grid, cv=10, verbose=0)
knn_clf.fit(X_titanic_train, y_titanic_train)

# Logistic Regression Classifier
print("Optimize Logistic Regression Classifier")
logr_clf_param_grid = [
  {"logr_clf__penalty": ["l2"],
   "logr_clf__C": np.logspace(-1, 3, 40),
   "logr_clf__multi_class": ["auto", "ovr", "multinomial"]}
]
logr_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ("logr_clf", LogisticRegression()),]),
    logr_clf_param_grid, cv=10, verbose=0)
logr_clf.fit(X_titanic_train, y_titanic_train)

# Quadratic Discriminant Classifier
print("Optimize Quadratic Discriminant Classifier")
qda_clf = Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ("qda_clf", QuadraticDiscriminantAnalysis()),])
qda_clf.fit(X_titanic_train, y_titanic_train)

# Random Forest Classifier
print("Optimize Random Forest Classifier")
rnd_forest_clf_param_grid = [
  {"rnd_forest_clf__criterion": ["gini", "entropy"], "rnd_forest_clf__n_estimators": [100, 200]},
]
rnd_forest_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ("rnd_forest_clf", RandomForestClassifier()),]),
    rnd_forest_clf_param_grid, cv=10, verbose=0)
rnd_forest_clf.fit(X_titanic_train, y_titanic_train)

# Ridge Regression Classifier
print("Optimize Ridge Regression Classifier")
ridge_cv_clf = Pipeline([
            ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
            ("scaler", StandardScaler()),
            ("ridge_clf", RidgeClassifierCV(alphas=np.logspace(-2, 2, 25), cv=5)),
        ])
ridge_cv_clf.fit(X_titanic_train, y_titanic_train)

# SVM Classifier
print("Optimize SVM Classifier")
param_grid = [
  {"svm_clf__C": np.logspace(-1, 2, 15), "svm_clf__kernel": ["linear"]},
  {"svm_clf__C": np.logspace(-1, 2, 15), "svm_clf__kernel": ["poly"], "svm_clf__degree": [1, 2, 3, 4, 5, 6]},
  {"svm_clf__C": np.logspace(-1, 2, 15), "svm_clf__kernel": ["rbf"]},
  {"svm_clf__C": np.logspace(-1, 2, 15), "svm_clf__kernel": ["rbf"]},
  {"svm_clf__C": np.logspace(-1, 2, 15), "svm_clf__kernel": ["sigmoid"]},
]
svm_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ("svm_clf", SVC()),]),
    param_grid, cv=3, verbose=0)
svm_clf.fit(X_titanic_train, y_titanic_train)

# Voting Classifier
vtg_clf = VotingClassifier([("knn_clf", knn_clf.best_estimator_), ("logr_clf", logr_clf.best_estimator_), ("qda_clf", qda_clf), ("rnd_forest_clf", rnd_forest_clf.best_estimator_), ("ridge_cv_clf", ridge_cv_clf), ("svm_clf", svm_clf.best_estimator_)])
vtg_clf.fit(X_titanic_train, y_titanic_train)
y_titanic_vtg_clf_pred = vtg_clf.predict(X_titanic_test)

# Score
print("Confusion Matrix")
print(confusion_matrix(y_titanic_test, y_titanic_vtg_clf_pred))
print("Precision Score")
print(precision_score(y_titanic_test, y_titanic_vtg_clf_pred))
print("Accuracy Score")
print(accuracy_score(y_titanic_test, y_titanic_vtg_clf_pred))
print("Recall Score")
print(recall_score(y_titanic_test, y_titanic_vtg_clf_pred))
print("F1 Score")
print(f1_score(y_titanic_test, y_titanic_vtg_clf_pred))


Titanic
Optimize K Nearest Neighbors Classifier
Optimize Logistic Regression Classifier
Optimize Quadratic Discriminant Classifier
Optimize Random Forest Classifier
Optimize Ridge Regression Classifier
Optimize SVM Classifier
Confusion Matrix
[[200  31]
 [ 35  91]]
Precision Score
0.7459016393442623
Accuracy Score
0.8151260504201681
Recall Score
0.7222222222222222
F1 Score
0.7338709677419354
